In [74]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# DDL statements.
drop_ddl = [
	"""DROP TABLE IF EXISTS "assertion";""",
	"""DROP TABLE IF EXISTS "retraction";""",    
	"""DROP TABLE IF EXISTS "statement";""",
	"""DROP TABLE IF EXISTS "property";""",
	"""DROP TABLE IF EXISTS "record";""",
	"""DROP TABLE IF EXISTS "transaction";""",
	"""DROP TABLE IF EXISTS "registry";"""
]

for sql in drop_ddl:
	cursor.execute(sql)

create_ddl = [
	"""
	CREATE TABLE IF NOT EXISTS "registry" (
		"id" INTEGER PRIMARY KEY AUTOINCREMENT
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "transaction" (
		"id" INTEGER PRIMARY KEY AUTOINCREMENT,
		"registry_id" INTEGER,
		"timestamp" DATETIME,
		FOREIGN KEY ("registry_id") REFERENCES "registry"("id")
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "record" (
		"id" INTEGER PRIMARY KEY AUTOINCREMENT,
		"registry_id" INTEGER,
		"identifier" TEXT,
		FOREIGN KEY ("registry_id") REFERENCES "registry"("id")
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "property" (
		"id" INTEGER PRIMARY KEY AUTOINCREMENT,
		"type_name" TEXT,
		"property_name" TEXT,
		"multi" INTEGER DEFAULT 'FALSE'
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "statement" (
		"id" INTEGER PRIMARY KEY AUTOINCREMENT,
		"record_id" INTEGER,
		"property_id" INTEGER,
		"transaction_id" INTEGER,
		"effective_from" DATETIME,
		FOREIGN KEY ("record_id") REFERENCES "record"("id"),
		FOREIGN KEY ("property_id") REFERENCES "property"("id"),
		FOREIGN KEY ("transaction_id") REFERENCES "transaction"("id")
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "assertion" (
		"statement_id" INTEGER PRIMARY KEY,
		"value" TEXT,
		FOREIGN KEY ("statement_id") REFERENCES "statement"("id")
	);
	""",
	"""
	CREATE TABLE IF NOT EXISTS "retraction" (
		"statement_id" INTEGER PRIMARY KEY,
		"assertion_id" INTEGER,
		FOREIGN KEY ("statement_id") REFERENCES "statement"("id"),
		FOREIGN KEY ("assertion_id") REFERENCES "assertion"("id")
	);
	""",
]

for sql in create_ddl:
	cursor.execute(sql)

connection.commit()
connection.close()


In [75]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Seed.
cursor.execute("""INSERT INTO "registry" ("id") VALUES (1);""")

cursor.execute("""INSERT INTO "transaction" ("registry_id", "timestamp") VALUES (1, '2020-01-01 10:00:00');""")

cursor.execute("""INSERT INTO "record" ("registry_id", "identifier") VALUES (1, '90110800000');""")

properties = [
    ('Person', 'first_name', 'TRUE'),
    ('Person', 'gender', 'FALSE'),
    ('Person', 'citizenship', 'TRUE'),
    ('Residency', 'residence', 'TRUE'),
]
cursor.executemany("""INSERT INTO "property" ("type_name", "property_name", "multi") VALUES (?, ?, ?);""", properties)

assertion_statements = [
    (1, 'John'),
    (1, 'Fitzgerald'),
    (2, 'Male'),
    (3, '{"isoCode": "BEL", "nisCode": "150"}'),
    (4, '{"address": {"bestIdentifier": {"id": "https://data.vlaanderen.be/id/adres/3706808", "namespace": "https://data.vlaanderen.be/id/adres", "objectId": "3706808", "versionId": "2023-11-01T15:46:20+01:00"}}}'),
]

for property_id, assertion_value in assertion_statements:
    cursor.execute(
      """
      INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from")
      VALUES (1, ?, 1, '2020-01-01 10:00:00');
      """, 
      (property_id,)
    )
    
    statement_id = cursor.lastrowid

    cursor.execute(
      """
      INSERT INTO "assertion" ("statement_id", "value")
      VALUES (?, ?);
      """, 
      (statement_id, assertion_value)
    )

connection.commit()
connection.close()

In [83]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Read all statements.
query = """
SELECT s.id AS statement_id, r.identifier AS record_identifier, p.type_name, p.property_name, COALESCE(a.value, 'not_applicable') AS value, COALESCE(rt.statement_id, 'not_applicable') AS retraction_statement_id, s.effective_from, t.timestamp AS transaction_timestamp 
FROM "statement" s
JOIN "transaction" t ON s.transaction_id = t.id
JOIN "record" r ON r.id = s.record_id
JOIN "property" p ON s.property_id = p.id
LEFT JOIN "assertion" a ON s.id = a.statement_id
LEFT JOIN "retraction" rt ON a.statement_id = rt.assertion_id
ORDER BY s.id;
"""

resultset_df = pd.read_sql(query, connection)
 
connection.close()

resultset_df

,statement_id,record_identifier,type_name,property_name,value,retraction_statement_id,effective_from,transaction_timestamp
0,1,90110800000,Person,first_name,John,not_applicable,2020-01-01 10:00:00,2020-01-01 10:00:00
1,2,90110800000,Person,first_name,Fitzgerald,6,2020-01-01 10:00:00,2020-01-01 10:00:00
2,3,90110800000,Person,gender,Male,not_applicable,2020-01-01 10:00:00,2020-01-01 10:00:00
3,4,90110800000,Person,citizenship,"{""isoCode"": ""BEL"", ""nisCode"": ""150""}",8,2020-01-01 10:00:00,2020-01-01 10:00:00
4,5,90110800000,Residency,residence,"{""address"": {""bestIdentifier"": {""id"": ""https:/...",not_applicable,2020-01-01 10:00:00,2020-01-01 10:00:00
5,6,90110800000,Person,first_name,not_applicable,not_applicable,2022-01-01 10:00:00,2022-01-01 10:00:00
6,7,90110800000,Person,first_name,Cornelius,not_applicable,2022-01-01 10:00:00,2022-01-01 10:00:00
7,8,90110800000,Person,citizenship,not_applicable,not_applicable,2023-01-01 10:00:00,2023-01-01 10:00:00


In [84]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Read the history of the Person.first_name property.
query = """
SELECT s.id AS statement_id, r.identifier AS record_identifier, p.type_name, p.property_name, COALESCE(a.value, 'not_applicable') AS value, COALESCE(rt.statement_id, 'not_applicable') AS retraction_statement_id, s.effective_from, t.timestamp AS transaction_timestamp 
FROM "statement" s
JOIN "transaction" t ON s.transaction_id = t.id
JOIN "record" r ON r.id = s.record_id
JOIN "property" p ON s.property_id = p.id
LEFT JOIN "assertion" a ON s.id = a.statement_id
LEFT JOIN "retraction" rt ON a.statement_id = rt.assertion_id
WHERE p.type_name = :type_name
AND p.property_name = :property_name
ORDER BY t.timestamp;
"""

resultset_df = pd.read_sql(query, connection, params={"type_name": "Person", "property_name": "first_name"})
 
connection.close()

resultset_df

,statement_id,record_identifier,type_name,property_name,value,retraction_statement_id,effective_from,transaction_timestamp
0,1,90110800000,Person,first_name,John,not_applicable,2020-01-01 10:00:00,2020-01-01 10:00:00
1,2,90110800000,Person,first_name,Fitzgerald,6,2020-01-01 10:00:00,2020-01-01 10:00:00
2,6,90110800000,Person,first_name,not_applicable,not_applicable,2022-01-01 10:00:00,2022-01-01 10:00:00
3,7,90110800000,Person,first_name,Cornelius,not_applicable,2022-01-01 10:00:00,2022-01-01 10:00:00


In [86]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Read current state of a record.
query = """
SELECT s.id AS statement_id, r.identifier AS record_identifier, p.type_name, p.property_name, a.value, s.effective_from, t.timestamp AS transaction_timestamp 
FROM "statement" s
JOIN "transaction" t ON s.transaction_id = t.id
JOIN "record" r ON r.id = s.record_id
JOIN "property" p ON s.property_id = p.id
LEFT JOIN "assertion" a ON s.id = a.statement_id
LEFT JOIN "retraction" rt ON a.statement_id = rt.assertion_id
WHERE record_id = :record_id
AND a.statement_id IS NOT NULL
AND rt.statement_id IS NULL
ORDER BY p.type_name, p.property_name;
"""

resultset_df = pd.read_sql(query, connection, params={"record_id": 1})

connection.close()

resultset_df

,statement_id,record_identifier,type_name,property_name,value,effective_from,transaction_timestamp
0,1,90110800000,Person,first_name,John,2020-01-01 10:00:00,2020-01-01 10:00:00
1,7,90110800000,Person,first_name,Cornelius,2022-01-01 10:00:00,2022-01-01 10:00:00
2,3,90110800000,Person,gender,Male,2020-01-01 10:00:00,2020-01-01 10:00:00
3,5,90110800000,Residency,residence,"{""address"": {""bestIdentifier"": {""id"": ""https:/...",2020-01-01 10:00:00,2020-01-01 10:00:00


In [79]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Update Person.first_name property.
# Transaction 2 - 1 statement.
cursor.execute("""INSERT INTO "transaction" ("registry_id", "timestamp") VALUES (1, '2022-01-01 10:00:00');""")

cursor.execute(
	"""
	INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from")
	VALUES 
	(1, 1, 2, '2022-01-01 10:00:00');
	"""
)
statement_id = cursor.lastrowid

cursor.execute(
	"""
	INSERT INTO "retraction" ("statement_id", "assertion_id")
	VALUES (?, ?)
	""",
	(statement_id, 2)
)

cursor.execute(
  """
  INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from")
  VALUES (1, 1, 2, '2022-01-01 10:00:00');
  """
)
    
statement_id = cursor.lastrowid

cursor.execute(
  """
  INSERT INTO "assertion" ("statement_id", "value")
  VALUES (?, ?);
  """, 
  (statement_id, 'Cornelius')
)

connection.commit()
connection.close()

In [80]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Retract (soft delete) Person.citizenship property.
# Transaction 3 - 1 statement.
cursor.execute("""INSERT INTO "transaction" ("registry_id", "timestamp") VALUES (1, '2023-01-01 10:00:00');""")

cursor.execute(
	"""
	INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from")
	VALUES (1, 3, 3, '2023-01-01 10:00:00');
	""")
statement_id = cursor.lastrowid

assertion_to_retract = 4

cursor.execute(
	"""
	INSERT INTO "retraction" ("statement_id", "assertion_id")
	VALUES (?, ?)
	""",
	(statement_id, assertion_to_retract)
)

connection.commit()
connection.close()